###  Loading the Boston housing dataset

In [1]:
from tensorflow.keras.datasets import boston_housing
from sklearn.preprocessing import normalize
from tensorflow.keras import layers, models
from sklearn.model_selection import KFold

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [4]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


We can see that we have 404 training samples and 102 test samples, each with 13 numerical features.

In [5]:
print(train_targets[0])

15.2


The targets are the median values of the houses at a location in 1000's of dollars.

### Preparing the data

In [6]:
normalize(train_data, axis=0) # Feature-wise normalization

array([[6.15626701e-03, 0.00000000e+00, 3.10983151e-02, ...,
        5.61526372e-02, 5.38018011e-02, 6.35435054e-02],
       [1.08742552e-04, 1.55659962e-01, 7.75547662e-03, ...,
        3.93068460e-02, 5.35957574e-02, 1.05566401e-02],
       [2.44669243e-02, 0.00000000e+00, 6.91498162e-02, ...,
        5.40134891e-02, 5.09036340e-02, 1.10658028e-02],
       ...,
       [1.73128932e-04, 6.60375595e-02, 2.31518169e-02, ...,
        4.51895032e-02, 4.91048185e-02, 2.65782931e-02],
       [1.07352925e-02, 0.00000000e+00, 7.48040553e-02, ...,
        3.93068460e-02, 3.55086465e-02, 5.35978606e-02],
       [7.18789766e-05, 1.13207245e-01, 1.11938653e-02, ...,
        4.17133876e-02, 5.10635890e-02, 1.48675509e-02]])

### Building the neural network

In [7]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) # Output layer. Single unit and no activation
    
    model.compile(loss='mse', # Computes the mean squared error between labels and predictions.
                  optimizer='rmsprop',
                  metrics=['mae']) # Computes the mean absolute error between labels and predictions.
    return model

### K-fold validation

In [8]:
K = 4
k_fold_validation = KFold(n_splits=K)
k_fold_validation.get_n_splits(train_data) # Returns the number of splitting iterations in the cross-validator

4

In [16]:
count = 1
for train_index, test_index in k_fold_validation.split(train_data):
    print('\n==========================\t{}\t============================\n'.format(count))
    print('Train indices: ',train_index)
    print('\nTest indices: ', test_index)
    count += 1
    model = build_model()
    model.fit(train_data[train_index])


==========================	1	============================

Train indices:  [101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262
 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280
 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298
 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316
 317 318 319 320 321 322 323 324 325 326 327 328

In [18]:
len(train_data) // k_fold_validation.get_n_splits()

101

In [ ]:
le